<a href="https://colab.research.google.com/github/julianovale/simulacao_python/blob/master/Ideias_Juliano_20200907_carla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import I, Matrix, symbols, Symbol, eye
from datetime import datetime
import numpy as np
import pandas as pd


In [14]:
'''
Rotas
'''
R1 = Matrix([[0,"L1_p1",0,0,0,0,0,0,0],[0,0,"L1_p2",0,0,0,0,0,0],[0,0,0,"L1_v1",0,0,0,0,0],[0,0,0,0,"L1_p3",0,0,0,0],[0,0,0,0,0,"L1_v2",0,0,0],[0,0,0,0,0,0,"L1_p4",0,0],
             [0,0,0,0,0,0,0,"L1_v3",0],[0,0,0,0,0,0,0,0,"L1_v4"],[0,0,0,0,0,0,0,0,0]])
R2 = Matrix([[0,"L2_p5",0,0,0,0,0,0,0],[0,0,"L2_p5",0,0,0,0,0,0],[0,0,0,"L2_v5",0,0,0,0,0],[0,0,0,0,"L2_p7",0,0,0,0],[0,0,0,0,0,"L2_v6",0,0,0],[0,0,0,0,0,0,"L2_p8",0,0],
             [0,0,0,0,0,0,0,"L2_v7",0],[0,0,0,0,0,0,0,0,"L2_v8"],[0,0,0,0,0,0,0,0,0]])
# R3 = Matrix([[0,"L3_p5",0,0,0,0,0],[0,0,"L3_p3",0,0,0,0],[0,0,0,"L3_v5",0,0,0],[0,0,0,0,"L3_p1",0,0],[0,0,0,0,0,"L3_v3",0],[0,0,0,0,0,0,"L3_v1"],[0,0,0,0,0,0,0]])

In [15]:
'''
Seções de bloqueio
'''
T1 = Matrix([[0, "p1"],["v1", 0]])
T2 = Matrix([[0, "p2"],["v2", 0]])
T3 = Matrix([[0, "p3"],["v3", 0]])
T4 = Matrix([[0, "p4"],["v4", 0]])
T5 = Matrix([[0, "p5"],["v5", 0]])
T6 = Matrix([[0, "p6"],["v6", 0]])
T7 = Matrix([[0, "p7"],["v7", 0]])
T8 = Matrix([[0, "p8"],["v8", 0]])

In [ ]:
def kronSum(A,B):
  m = np.size(A,1)
  n = np.size(B,1)
  A = np.kron(A, np.eye(n))
  B = np.kron(np.eye(m),B)
  return A + B

In [5]:
momento_inicio = datetime.now() 
'''
Algebra de rotas
'''
rotas = kronSum(R1,R2)
# rotas = kronSum(rotas,R3)

'''
Algebra de seções 
'''
secoes = kronSum(T1,T2)
secoes = kronSum(secoes,T3)
secoes = kronSum(secoes,T4)
secoes = kronSum(secoes,T5)
secoes = kronSum(secoes,T6)
secoes = kronSum(secoes,T7)
secoes = kronSum(secoes,T8)
'''
Algebra de sistema
'''
sistema = np.kron(rotas, secoes)
# calcula tempo de processamento
tempo_processamento = datetime.now() - momento_inicio

In [6]:
sistema = pd.DataFrame(data=sistema,index=list(range(1,np.size(sistema,0)+1)), columns=list(range(1,np.size(sistema,1)+1)))

In [7]:
sistema.shape

(20736, 20736)

In [9]:
print(tempo_processamento)

0:07:41.578004


In [11]:
sistema

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,20697,20698,20699,20700,20701,20702,20703,20704,20705,20706,20707,20708,20709,20710,20711,20712,20713,20714,20715,20716,20717,20718,20719,20720,20721,20722,20723,20724,20725,20726,20727,20728,20729,20730,20731,20732,20733,20734,20735,20736
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20732,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20733,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
momento_inicio = datetime.now() 
colunas = ['denode', 'paranode', 'aresta']
grafo = pd.DataFrame(columns=colunas)
r = 1
c = 1
for j in range(np.size(sistema,0)):
  for i in range(np.size(sistema,0)):
    if sistema.loc[r,c]==0 and c < np.size(sistema,0):
      c += 1
    elif c < np.size(sistema,0):
      grafo.loc[len(grafo)+1] = (r, c, sistema.loc[r,c])  
      c += 1
    else:
      c = 1
      r += 1
tempo_processamento = datetime.now() - momento_inicio
print(tempo_processamento)

KeyboardInterrupt: ignored

In [ ]:
grafo

In [ ]:
grafo['aresta'] = grafo['aresta'].astype('str')
grafo

In [ ]:
new = grafo["aresta"].str.split("*", n = -1, expand = True) 
grafo["aresta"]=new[1]
grafo["semaforo_secao"]=new[2]
new = grafo["aresta"].str.split("_", n = -1, expand = True) 
grafo["trem"]=new[0]
grafo["semaforo_trem"]=new[1]
grafo['coincide'] = np.where(grafo['semaforo_secao']==grafo['semaforo_trem'], True, False)
grafo

In [ ]:
grafo = pd.DataFrame(data=grafo)

In [ ]:
grafo.drop(grafo[grafo.coincide == False].index, inplace=True)
grafo.reset_index(drop = True)
grafo.index = np.arange(1, len(grafo) + 1)
grafo

In [ ]:
# Passo 1:
alcancavel = [1]
r = 1
for i in range(np.size(grafo,0)):
  de = grafo.loc[r]['denode']
  para = grafo.loc[r]['paranode']
  if (de in alcancavel):
    alcancavel.append(para)
    r = r + 1
  else:
    r = r + 1

In [ ]:
alcancavel.sort()

In [ ]:
grafo = grafo[grafo.denode.isin(alcancavel)]
grafo

In [ ]:
grafo.reset_index(drop = True)
grafo.index = np.arange(1, len(grafo) + 1)
grafo

In [ ]:
# Passo 2:
ultimo = alcancavel[-1]
alcancavel = [ultimo]
r = np.size(grafo,0)
for i in range(np.size(grafo,0),1,-1):
  para = grafo.loc[r]['paranode']
  de = grafo.loc[r]['denode']
  if (para in alcancavel):
    alcancavel.append(de)
    r = r - 1
  else:
    r = r - 1


In [ ]:
grafo = grafo[grafo.paranode.isin(alcancavel)]
grafo

In [ ]:
grafo.reset_index(drop = True)
grafo.index = np.arange(1, len(grafo) + 1)
grafo


In [ ]:
grafo.to_csv('grafo.csv', sep=";")
from google.colab import files
files.download('grafo.csv')

In [ ]:
# Passo 3

'''
contra = [["v1","p1"],["v2","p2"],["v3","p3"],["v4","p4"],["v5","p5"]]
contra = pd.DataFrame(contra,columns=["operador_v","operador_p"])
v = grafo
v = v.loc[v['semaforo_trem'].isin(contra['operador_v'])]
v = v.drop(columns=["aresta", "semaforo_secao", "coincide"])
v = v.rename(columns={"denode": "noh_anterior","semaforo_trem": "operador_v", "paranode":"denode"})
v = v.merge(contra, on="operador_v")
v = grafo.merge(v, on="denode")
v['coincide'] = np.where(v['semaforo_secao']==v['operador_p'], True, False)
v.drop(v[v.coincide == False].index, inplace=True)
v.reset_index(drop = True)
v.index = np.arange(1, len(v) + 1)
v = v.drop(columns=["aresta","semaforo_secao", "coincide", "operador_p"])
v = v.rename(columns={"trem_x": "trem_atual", "semaforo_trem":"semaforo_atual", "trem_y": "trem_anterior", "operador_v": "semaforo_anterior"})
nohs_sincroniza = v
'''

In [ ]:
# nohs_sincroniza

In [ ]:
# Passo 4

In [ ]:
'''grafo.to_csv('grafo.csv', sep=";")
from google.colab import files
files.download('grafo.csv')'''